Урок 5. Парсинг данных. Scrapy. Начало

###Вариант I
Написать программу, которая собирает входящие письма из своего или тестового почтового ящика и сложить данные о письмах в базу данных (от кого, дата отправки, тема письма, текст письма полный)
Логин тестового ящика: study.ai_172@mail.ru
Пароль тестового ящика: NextPassword172#
Если тестовый почтовый ящик не работает, то нужно создать свой почтовый ящик и переслать на него минимум любых рекламных 50 сообщений.



In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
import time
from pprint import pprint
from pymongo import MongoClient

client = MongoClient('localhost', 27017)
db = client['letters_db']
collection = db.ai_172_collection

chrome_options = Options()
chrome_options.add_argument('start-maximized')

driver = webdriver.Chrome(options=chrome_options)

driver.get('https://mail.ru/')

assert "Mail.ru" in driver.title

elem = driver.find_element_by_id('mailbox:login')
elem.send_keys('study.ai_172@mail.ru')
elem.send_keys(Keys.RETURN)

elem = driver.find_element_by_id('mailbox:password')
elem.send_keys('NextPassword172')
elem.send_keys(Keys.RETURN)

driver.implicitly_wait(5)
first_letter = driver.find_element_by_class_name('js-letter-list-item')
driver.get(first_letter.get_attribute('href'))

assert "Почта Mail.ru" in driver.title

letters_total = int(driver.find_element_by_class_name("nav__item_active").get_attribute('title').split(' ')[1])
print(letters_total)
letters_quantity = int(input(f'Всего в ящике писем: {letters_total}. \n'
                             f'Сколько писем нужно просканировать начиная с самого позднего? \n'))

# letters_set = []
num = 0
i = 1
while i <= letters_quantity:
    item = {}

    sender = driver.find_element_by_xpath("//span[contains(@class, 'letter-contact')]").get_attribute('title')
    subject = driver.find_element_by_xpath("//h2[contains(@class, 'thread__subject')]").text
    date = driver.find_element_by_class_name("letter__date").text
    body = driver.find_element_by_class_name("letter__body").text  # текст сырой, необработанный, весь
    _id = driver.find_element_by_class_name('thread__letter').get_attribute('data-id')

    item['sender'] = sender
    item['subject'] = subject
    item['date'] = date
    item['body'] = body
    item['_id'] = _id

    collection.insert_one(item)
    num += 1
    i += 1

    if i > letters_quantity:
        break
    else:
        next_button = WebDriverWait(driver, 5).until(
            EC.element_to_be_clickable((By.XPATH, "//div[contains(@class, 'portal-menu-element_next')]"))
        )

        next_button.click()

    time.sleep(1)

driver.quit()

# pprint(letters_set)
print(f'Всего писем просканировано: {num}')

for item in collection.find({}):
    pprint(item)
print(f'Всего в базе данных {collection.count_documents({})} писем')
collection.drop() 

###Вариант II
2) Написать программу, которая собирает товары «В тренде» с сайта техники mvideo и складывает данные в БД.

In [ ]:
from selenium import webdriver
from selenium.common import exceptions
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from pprint import pprint
from selenium.webdriver.common.action_chains import ActionChains
import time
from pymongo import MongoClient

client = MongoClient('localhost', 27017)
db = client['mvideo_db']
collection = db.hits_collection

chrome_options = Options()
chrome_options.add_argument('start-maximized')

driver = webdriver.Chrome(options=chrome_options)

driver.get('https://www.mvideo.ru/')

assert "М.Видео" in driver.title

hits = driver.find_element_by_xpath(
    "//div[contains(text(),'Хиты продаж')]/ancestor::div[@data-init='gtm-push-products']")

while True:
    try:
        next_button = WebDriverWait(hits, 5).until(
            EC.presence_of_element_located((By.XPATH, ".//a[contains(@class, 'next-btn')][not(contains(@class, 'disabled'))]"))
        )

        next_button.click()
        time.sleep(1)

    except exceptions.TimeoutException:
        break

hits = hits.find_elements_by_xpath(".//li[contains(@class, 'gallery-list-item')]")

# hits_set = []
num = 0
for hit in hits:
    item = {}

    data_product_info = hit.find_element_by_xpath(".//a[@class='sel-product-tile-title']").get_attribute('data-product-info').replace('\n', '').replace('\t', '')
    data_product_info = data_product_info.replace('{', '').replace('}', '').replace('"', '').replace(': ', ',').split(',')
    product_info = {}
    for i in range(0, len(data_product_info), 2):
        product_info[data_product_info[i]] = data_product_info[i + 1]

    url = hit.find_element_by_xpath(".//a[@class='sel-product-tile-title']").get_attribute('href')

    item['_id'] = product_info.get('productId')
    item['title'] = product_info.get('productName')
    item['price'] = float(product_info.get('productPriceLocal'))
    item['url'] = url

    # hits_set.append(item)
    collection.insert_one(item)
    num += 1

driver.quit()

# pprint(hits_set)
print(f'Всего Хитов продаж найдено: {num}')

for item in collection.find({}):
    pprint(item)
print(f'Всего в базе данных {collection.count_documents({})} товаров')
collection.drop()  